In [1]:
import pandas as pd

YOLO_CARS_CLASSES = ["car", "bus", "truck"]

# YOLO - only cars

for wildlife it did not know the names of animals

In [8]:
path_to_csv = 'datasets_generated_ready_models/SD_XL/detected_objects_cars.csv'
df_sdxl_cars = pd.read_csv(path_to_csv)
df_sdxl_cars.head(3)

,Unnamed: 0,image_path,predicted_class,confidence
0,0,/content/SD_XL/cars_zipped.zip/cars_img_0.png,train,0.71
1,1,/content/SD_XL/cars_zipped.zip/cars_img_0.png,car,0.33
2,2,/content/SD_XL/cars_zipped.zip/cars_img_1.png,car,0.80


In [9]:
path_to_csv = 'datasets_generated_ready_models/DALLE3/detected_objects_cars.csv'
df_dalle_cars = pd.read_csv(path_to_csv)
df_dalle_cars.head(3)

,Unnamed: 0,image_path,predicted_class,confidence
0,0,/content/DALLE3/dalle_cars_zipped.zip/car_img0...,car,0.94
1,1,/content/DALLE3/dalle_cars_zipped.zip/car_img1...,car,0.96
2,2,/content/DALLE3/dalle_cars_zipped.zip/car_img1...,potted plant,0.68


Filter-out non-car objects, count unique image_paths.

In [21]:
df_sdxl_cars['class'] = df_sdxl_cars['predicted_class'].apply(lambda x: 'car' if x in YOLO_CARS_CLASSES else 'other')
df_dalle_cars['class'] = df_dalle_cars['predicted_class'].apply(lambda x: 'car' if x in YOLO_CARS_CLASSES else 'other')

df_sdxl_cars.head(3)

,Unnamed: 0,image_path,predicted_class,confidence,class
0,0,/content/SD_XL/cars_zipped.zip/cars_img_0.png,train,0.71,other
1,1,/content/SD_XL/cars_zipped.zip/cars_img_0.png,car,0.33,car
2,2,/content/SD_XL/cars_zipped.zip/cars_img_1.png,car,0.80,car


In [22]:
# count the number of cars detected by each model
# 1. filter non cars
# 2. count the number of unique image_path
df_sdxl_cars_filtered = df_sdxl_cars[df_sdxl_cars['class'] == 'car']
df_dalle_cars_filtered = df_dalle_cars[df_dalle_cars['class'] == 'car']

print(f"Number of cars detected by SD_XL: {df_sdxl_cars_filtered['image_path'].nunique()}")
print(f"Number of cars detected by DALLE3: {df_dalle_cars_filtered['image_path'].nunique()}")

Number of cars detected by SD_XL: 47
Number of cars detected by DALLE3: 46


# Hypothesis testing

$\phi(x) = 1$ if at least one car was detected.

In [33]:
Sn_SDXL_cars = df_sdxl_cars_filtered['image_path'].nunique() / len(df_sdxl_cars['image_path'].unique())
Sn_DALLE_cars = df_dalle_cars_filtered['image_path'].nunique() / len(df_dalle_cars['image_path'].unique())

Sn_SDXL_cars, Sn_DALLE_cars

(0.94, 0.92)

In [41]:
import numpy as np

def add_results_row(results, experiment_name, N, Sn, p0_to_test=[0.5, 0.6, 0.7, 0.8, 0.9]):
    results["Method"].append("YOLO")
    results["Experiment"].append(experiment_name)
    results["N"].append(N)
    results["Sn"].append(Sn)
    for p0 in p0_to_test:
        if Sn <= p0:
            results[f"p0={p0}"].append(0)
            continue
        delta = np.exp(-2*N*(Sn-p0)**2)
        results[f"p0={p0}"].append(round(1-delta, 3))
    p0_for_CI_095 = Sn - np.sqrt(np.log(0.05)/(-2*N))
    results["p0 for confidence=0.95"].append(round(p0_for_CI_095, 3))

results = {
    "Method": [],
    "Experiment": [],
    "N": [],
    "Sn" : [],
    "p0=0.5": [],
    "p0=0.6": [],
    "p0=0.7": [],
    "p0=0.8": [],
    "p0=0.9": [],
    "p0 for confidence=0.95": [],
}

add_results_row(results, "SDXL cars", 50, Sn_SDXL_cars)
add_results_row(results, "DALLE cars", 50, Sn_DALLE_cars)

df_results = pd.DataFrame(results)
df_results.to_csv("confidence_matrix_YOLO_SDXL_DALLE.csv", index=False)
df_results

,Method,Experiment,N,Sn,p0=0.5,p0=0.6,p0=0.7,p0=0.8,p0=0.9,p0 for confidence=0.95
0,YOLO,SDXL cars,50,0.94,1.0,1.0,0.997,0.859,0.148,0.767
1,YOLO,DALLE cars,50,0.92,1.0,1.0,0.992,0.763,0.039,0.747
